In [112]:
%matplotlib qt

import matplotlib.pyplot as plt
import numpy as np
import cv2 
from typing import Sequence
from camera_models import *  # our package

DECIMALS = 2  # how many decimal places to use in print


from utils import *

In [113]:
def to_inhomogeneus(X: np.ndarray) -> np.ndarray:
    if X.ndim > 1:
        raise ValueError("x must be one-dimensional.")

    return (X / X[-1])[:-1]


def to_homogeneus(X: np.ndarray) -> np.ndarray:
    if X.ndim > 1:
        raise ValueError("X must be one-dimensional.")

    return np.hstack([X, 1])

def to_homogeneus_arr(X: np.ndarray) -> np.ndarray:
    one_matrix = np.ones((X.shape[0],1))

    out_value = np.concatenate((X, one_matrix), axis=1)

    return out_value

def _get_roll_matrix(theta_x: float = 0.0) -> np.ndarray:
    Rx = np.array(
        [
            [1.0, 0.0, 0.0],
            [0.0, np.cos(theta_x), -np.sin(theta_x)],
            [0.0, np.sin(theta_x), np.cos(theta_x)],
        ]
    )
    return Rx


def _get_pitch_matrix(theta_y: float = 0.0) -> np.ndarray:
    Ry = np.array(
        [
            [np.cos(theta_y), 0.0, np.sin(theta_y)],
            [0.0, 1.0, 0.0],
            [-np.sin(theta_y), 0.0, np.cos(theta_y)],
        ]
    )
    return Ry


def _get_yaw_matrix(theta_z: float = 0.0) -> np.ndarray:
    Rz = np.array(
        [
            [np.cos(theta_z), -np.sin(theta_z), 0.0],
            [np.sin(theta_z), np.cos(theta_z), 0.0],
            [0.0, 0.0, 1.0],
        ]
    )
    return Rz


#my and mx correspond to the pixel density in the x and y direction
def get_calibration_matrix(
    f: float,
    px: float = 0.0,
    py: float = 0.0,
    mx: float = 1.0,
    my: float = 1.0,
) -> np.ndarray:
    K = np.diag([mx, my, 1]) @ np.array([[f, 0.0, px], [0.0, f, py], [0.0, 0.0, 1.0]])
    return K

def get_rotation_matrix(
    theta_x: float = 0.0, theta_y: float = 0.0, theta_z: float = 0.0
) -> np.ndarray:
    # Roll
    Rx = _get_roll_matrix(theta_x)
    # Pitch
    Ry = _get_pitch_matrix(theta_y)
    # Yaw
    Rz = _get_yaw_matrix(theta_z)
    return Rz @ Ry @ Rx

def get_translation_matrix(Tx: float,Ty: float,Tz: float):
    return np.array([[Tx],[Ty],[Tz]])

def get_extrinsic_matrix(rot: np.array((3,3)), trans:np.array((3,1))):
    E = np.concatenate((rot,trans), axis = 1)
    return E

def get_projection_matrix(
    f: float,
    px: float = 0.0,
    py: float = 0.0,
    tx : float = 0.0,
    ty : float = 0.0,
    tz : float = 0.0,
    theta_x: float = 0.0,
    theta_y: float = 0.0,
    theta_z: float = 0.0,
    mx: float = 1.0,
    my: float = 1.0,
) -> np.ndarray:
    K = get_calibration_matrix(f=f, px=px, py=py, mx=mx, my=my)
    #print("Calibration matrix: \n",K)
    R = get_rotation_matrix(theta_x=theta_x, theta_y=theta_y, theta_z=theta_z)
    #print("Rotation matrix: \n",R)
    T = get_translation_matrix(tx,ty,tz)
    #print("Translation matrix: \n",T)
    E = get_extrinsic_matrix(rot = R,trans = T)
    P = K @ E
    
    return P

def get_plucker_matrix(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    A = to_homogeneus(A)
    B = to_homogeneus(B)
    L = A.reshape(-1, 1) * B.reshape(1, -1) - B.reshape(-1, 1) * A.reshape(1, -1)
    return L



def normalization(data, dim):
    '''
    Normalization of coordinates (centroid to the origin and mean distance of sqrt(2 or 3).
    Inputs:
    data: the data to be normalized (directions at different columns and points at rows) -> (nº points, nº coordinates)
    Outputs:
    T : the transformation matrix (translation plus scaling)
    '''

    N = data.shape[0]
    
    if dim == 2:                                                     # 2D points
        x, y = data[:, 0], data[:, 1]                               # x and y coordinates
        x_mean, y_mean = x.mean(), y.mean()                         # mean 
        x_par = (x-x_mean)**2                                       # x part
        y_par = (y-y_mean)**2                                       # y part

        d = (1/N)* np.sum(np.sqrt(x_par + y_par))                   # d
        s_xy = np.sqrt(2)/d                                         # sqrt(2)/d

        T = np.zeros((3,3))                                         # normalization matrix for 2D points
        T[0,0] = s_xy
        T[0,2] = -x_mean*s_xy
        T[1,1] = s_xy
        T[1,2] = -y_mean*s_xy
        T[-1,-1] = 1

        coord_n = np.dot(data,np.transpose(T))                          # normalized image coordinates

    elif(dim== 3):                                                  # 3D points
        X, Y, Z = data[:, 0], data[:, 1], data[:,2]                 # x and y coordinates
        X_mean, Y_mean,Z_mean = X.mean(), Y.mean(),Z.mean()         # mean 
        X_par = (X-X_mean)**2                                       # x part
        Y_par = (Y-Y_mean)**2                                       # y part
        Z_par = (Z-Z_mean)**2                                       # z part

        D = (1/N)* np.sum(np.sqrt(X_par + Y_par + Z_par))           # d
        s_xyz = np.sqrt(3)/D                                        # sqrt(2)/d

        T = np.zeros((4,4))                                         # normalization matrix for 3D points
        T[0,0] = s_xyz
        T[0,3] = -X_mean*s_xyz
        T[1,1] = s_xyz
        T[1,3] = -Y_mean*s_xyz
        T[2,2] = s_xyz
        T[2,3] = -Z_mean*s_xyz
        T[-1,-1] = 1

        coord_n = np.dot(data,T.T)                          # normalized world coordinates

    else:
        raise ValueError('Dataset must be a collection of 2D or points')

    return T,coord_n

def denormalization(H_norm,Txy,Txyz):
    H = np.linalg.pinv(Txy).dot((H_norm).dot(Txyz))
    H = H / H[-1, -2]

    return H


def create_chessboard(patternsize_x,patternsize_y):
    """
    Creates chessboard with pattern dimensions
    patternsize_x[in] - size x of the chessboard
    patternsize_y[in] - size y of the chessboard
    world_pts_arr[out] - meshgrid points
    """
    pattern_size_x,pattern_size_y = (patternsize_x,patternsize_y)

    X = np.linspace(-pattern_size_x/2, pattern_size_x/2,pattern_size_x + 1)
    Y = np.linspace(-pattern_size_y/2, pattern_size_y/2,pattern_size_y + 1)

    zdata = np.zeros((patternsize_x + 1,patternsize_y + 1))
    xdata, ydata = np.meshgrid(X, Y)

    xdata_ = xdata.flatten()
    ydata_ = ydata.flatten()
    zdata_ = zdata.flatten()
    onedata = np.ones((zdata_.shape[0])).flatten()
    # homogeneous coordinates
    world_pts =([xdata_,ydata_,zdata_,onedata])
    world_pts_arr = np.array(world_pts).T

    return world_pts_arr

def get_chessboard_rot_trans(world_pts_arr,rx, ry, rz, tx,ty,tz):
    """
    rotates and translates the chessboard
    world_pts_arr[in] - chessboard points in homogenous coordinates[4,1]
    rx[in] - rotation on the x axis
    ry[in] - rotation on the y axis
    rz[in] - rotation on the z axis
    tx[in] - translation on the x axis
    ty[in] - translation on the y axis
    tz[in] - translation on the z axis
    world_trans_arr[out] - output chessboard rotation and translation 
    """ 
    
    Rx =get_rotation_matrix(rx,ry,rz)
    Trans = np.array([[tx],[ty],[tz]])
    E = np.concatenate((Rx,Trans),axis=1)

    world_trans_arr = np.dot(world_pts_arr,E.T)

    return world_trans_arr

In [114]:
F = 1                                       # focal length( in mm )
sensor_size = np.array([11,7])              # sensor size(mm)
PX= sensor_size[0]/2.0                      # principal point x-coordinate
PY= sensor_size[1]/2.0                      # principal point y-coordinate
IMAGE_HEIGTH = sensor_size[1]
IMAGE_WIDTH = sensor_size[0]
THETA_X = -np.pi / 2                         # roll angle
THETA_Z = np.pi/2                             # yaw angle

C = np.array([8,2,4])                    # camera centre


In [115]:
def get_image_points(rand_points,px,py,thetax = 0, thetay = 0, thetaz = 0,trans_x = 0,trans_y = 0,trans_z = 0,F = F):

    K = get_calibration_matrix(F, px=px, py=py)                                             # calibration matrix
    P = get_projection_matrix(F, px=px,py =py,theta_x = thetax,theta_y = thetay,theta_z = thetaz,tx = trans_x,ty = trans_y, tz = trans_z)                        # projection matrix
    #print("\nCalibration matrix (K):\n", K)
    #print("\nProjection matrix (P):\n", P)

    x = []

    for i in range(rand_points.shape[1]):
        Xh = to_homogeneus(rand_points[:,i])
        xh = P @ Xh
        Xx = to_inhomogeneus(xh)
        x.append(Xx)

    x_arr = np.array(x)

    return x_arr, rand_points.T

In [116]:
def SVD(M):
    U, S, Vh = np.linalg.svd(M)

    L = Vh[-1,:] / Vh[-1,-2]
    #Camera projection matrix:
    PEst = L.reshape(3,3+1)
    #print("Camera Matrix:\n",PEst)

    return PEst

In [117]:
def DLT(x_arr,X_arr):

    worldm, worldn = X_arr.shape
    imagem, imagen = x_arr.shape
    
    #3D DLT
    A = []
    world_pts = X_arr
    image_pts = x_arr

    for i in range(worldm):
        A.append([-world_pts[i,0],-world_pts[i,1],-world_pts[i,2],-1,0,0,0,0,image_pts[i,0]*world_pts[i,0],image_pts[i,0]*world_pts[i,1],image_pts[i,0]*world_pts[i,2],image_pts[i,0]])
        A.append([0,0,0,0,-world_pts[i,0],-world_pts[i,1],-world_pts[i,2],-1,image_pts[i,1]*world_pts[i,0],image_pts[i,1]*world_pts[i,1],image_pts[i,1]*world_pts[i,2],image_pts[i,1]])

    # pass the list A as an array
    M = np.asarray(A).reshape(worldm*2,12)
    #print(M.shape)
    
    PEst = SVD(M)
    return PEst

In [118]:
X = np.array(
 [[-2.00000000e+00,  2.26794919e+00,  2.00000000e+00],
 [-1.50000000e+00 , 3.13397460e+00 , 2.00000000e+00],
 [-1.00000000e+00 , 4.00000000e+00 , 2.00000000e+00],
 [-5.00000000e-01 , 4.86602540e+00 , 2.00000000e+00],
 [ 2.22044605e-16 , 5.73205081e+00 , 2.00000000e+00],
 [-2.00000000e+00 , 2.26794919e+00 , 3.00000000e+00],
 [-1.50000000e+00 , 3.13397460e+00 , 3.00000000e+00],
 [-1.00000000e+00 , 4.00000000e+00 , 3.00000000e+00],
 [-5.00000000e-01 , 4.86602540e+00 , 3.00000000e+00],
 [ 2.22044605e-16 , 5.73205081e+00 , 3.00000000e+00],
 [-2.00000000e+00 , 2.26794919e+00 , 4.00000000e+00],
 [-1.50000000e+00 , 3.13397460e+00 , 4.00000000e+00],
 [-1.00000000e+00 , 4.00000000e+00 , 4.00000000e+00],
 [-5.00000000e-01 , 4.86602540e+00 , 4.00000000e+00],
 [ 2.22044605e-16 , 5.73205081e+00 , 4.00000000e+00],
 [-2.00000000e+00 , 2.26794919e+00 , 5.00000000e+00],
 [-1.50000000e+00 , 3.13397460e+00 , 5.00000000e+00],
 [-1.00000000e+00 , 4.00000000e+00 , 5.00000000e+00],
 [-5.00000000e-01 , 4.86602540e+00 , 5.00000000e+00],
 [ 2.22044605e-16 , 5.73205081e+00 , 5.00000000e+00],
 [-2.00000000e+00 , 2.26794919e+00 , 6.00000000e+00],
 [-1.50000000e+00 , 3.13397460e+00 , 6.00000000e+00],
 [-1.00000000e+00 , 4.00000000e+00 , 6.00000000e+00],
 [-5.00000000e-01 , 4.86602540e+00 , 6.00000000e+00],
 [ 2.22044605e-16 , 5.73205081e+00 , 6.00000000e+00]])

In [119]:

def get_plane(X):
    x_plane, y_plane, z_plane = X[:,0],X[:,1],X[:,2]
    x_surface = np.array([[x_plane[0],x_plane[4]],[x_plane[5],x_plane[9]],[x_plane[10],x_plane[14]],[x_plane[15],x_plane[19]],[x_plane[20],x_plane[24]]])
    y_surface = np.array([[y_plane[0],y_plane[4]],[y_plane[5],y_plane[9]],[y_plane[10],y_plane[14]],[y_plane[15],y_plane[19]],[y_plane[20],y_plane[24]]])
    z_surface = np.array([[z_plane[0],z_plane[4]],[z_plane[5],z_plane[9]],[z_plane[10],z_plane[14]],[z_plane[15],z_plane[19]],[z_plane[20],z_plane[24]]])

    return x_surface, y_surface,z_surface

In [120]:

x_arr, X_arr = get_image_points(X.T,PX,PY,-np.pi/2,np.pi/2,trans_x= C[0],trans_y= C[1],trans_z= C[2],F = 1)
#print(X_arr)
print(x_arr)
X_h = to_homogeneus_arr(X_arr)
x_h = to_homogeneus_arr(x_arr)

Txyz, X_h_n = normalization(X_h, 3)
Txy, x_h_n = normalization(x_h, 2)

P_n = DLT(x_h,X_h)

#PEst = denormalization(P_n,Txy,Txyz)

print("Camera Matrix:\n",P_n)

n_points = X_h.shape[0]
xEst = np.zeros((n_points,2))

for j in range(n_points):
    X_pts = to_homogeneus(X_arr[j])
    x_aux  = np.dot(P_n,X_pts)
    xEst[j,:] = to_inhomogeneus(x_aux)
    
err_list = np.sqrt(np.mean(np.sum(np.square(x_arr-xEst), 1)))



print("Geometric error:",err_list)

[[6.4553418  4.16666667]
 [6.38473189 4.22727273]
 [6.3        4.3       ]
 [6.1964388  4.38888889]
 [6.0669873  4.5       ]
 [6.4553418  4.33333333]
 [6.38473189 4.40909091]
 [6.3        4.5       ]
 [6.1964388  4.61111111]
 [6.0669873  4.75      ]
 [6.4553418  4.5       ]
 [6.38473189 4.59090909]
 [6.3        4.7       ]
 [6.1964388  4.83333333]
 [6.0669873  5.        ]
 [6.4553418  4.66666667]
 [6.38473189 4.77272727]
 [6.3        4.9       ]
 [6.1964388  5.05555556]
 [6.0669873  5.25      ]
 [6.4553418  4.83333333]
 [6.38473189 4.95454545]
 [6.3        5.1       ]
 [6.1964388  5.27777778]
 [6.0669873  5.5       ]]
Camera Matrix:
 [[-2.27163289e+16 -4.13023650e+15  2.43125000e+01  1.23907189e+17]
 [-1.44558428e+16  8.52658237e+08  4.13024039e+15  6.60838414e+16]
 [-4.13024084e+15  2.60510787e+08  1.00000000e+00  1.65209601e+16]]
Geometric error: 7.246142549764706e-15


In [121]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(x_arr[:,0],x_arr[:,1], color = 'r',ls = "None", marker = ".", label = 'Original')
ax.set_title("Point estimation")
ax.set_xlabel("X axis")
ax.set_ylabel("Y axis")
ax.grid()
ax.plot(xEst[:,0],xEst[:,1], color = 'b',ls = "None", marker = ".", label = 'Estimation')
ax.legend()
plt.show()

In [122]:
R = get_rotation_matrix(theta_x=THETA_X, theta_z=THETA_Z)
world_origin = np.zeros(3)
dx, dy, dz = np.eye(3)
world_frame = ReferenceFrame(
    origin=world_origin, 
    dx=dx, 
    dy=dy,
    dz=dz,
    name="World",
)
camera_frame = ReferenceFrame(
    origin=C, 
    dx=R @ dx, 
    dy=R @ dy,
    dz=R @ dz,
    name="Camera",
)
Z = PrincipalAxis(
    camera_center=camera_frame.origin,
    camera_dz=camera_frame.dz,
    f=F,
)
image_frame = ReferenceFrame(
    origin=Z.p - camera_frame.dx * PX - camera_frame.dy * PY, 
    dx=R @ dx, 
    dy=R @ dy,
    dz=R @ dz,
    name="Image",
)
image_plane = ImagePlane(
    origin=image_frame.origin, 
    dx=image_frame.dx, 
    dy=image_frame.dy, 
    heigth=IMAGE_HEIGTH,
    width=IMAGE_WIDTH,
)

In [123]:
G = []
L = []
pi = []
x = []
#X = np.array([[6, 2,5],[6,4,5]])                        # Create a random point
for i in range(X.shape[0]):
    G.append(GenericPoint(X[i], name="X"))                   
    L.append(get_plucker_matrix(C, X[i]))
    X1 = image_frame.origin
    X2 = X1 + image_frame.dx
    X3 = X1 + image_frame.dy
    pi.append(get_plane_from_three_points(X1, X2, X3))
    x.append(to_inhomogeneus(L[i] @ pi[i]))

    """
    print(f"X:\n{X}")
    print(f"\nL:\n{L[i].round(DECIMALS)}")
    print(f"\nX1:\n{X1}")
    print(f"\nX2:\n{X2}")
    print(f"\nX3:\n{X3}")
    print(f"\npi:\n{pi[i].round(DECIMALS)}")
    print(f"\nx:\n{x[i].round(DECIMALS)}")
    """
x_arr = np.array(x)

In [124]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(projection="3d")
ax.text(*C, "C")
world_frame.draw3d()
camera_frame.draw3d()
image_frame.draw3d()
Z.draw3d()
image_plane.draw3d()
for i in range(len(G)):
    G[i].draw3d(pi[i], C=C)
    x_surface, y_surface, z_surface = get_plane(X)
    ax.plot_surface(x_surface,y_surface,z_surface)

ax.view_init(elev=30.0, azim=30.0)

#ax.set_title(f"Representation of a Generic Point in 3D (X = {X})")
ax.set_xlabel("X-axis")
ax.set_ylabel("Y-axis")
ax.set_zlabel("Z-axis")
plt.tight_layout()
plt.show()

## Multiple planes

In [125]:
# chessboard print
chess_pts = []
world_pts = create_chessboard(4,4)
rotangles = [[np.pi/2,0,np.pi/2],[-np.pi/3,0,np.pi/2],[np.pi/3,0,np.pi/2],[np.pi/2,0,np.pi/3],[np.pi/2,0,-np.pi/3]]
transangles =  [[1,6,2],[1,4,2],[1,2,2],[1,0,2],[1,-2,2],[1,6,4],[1,4,4],[1,2,4],[1,0,4],[1,-2,4]]

for i in range(len(transangles)):
    for j in range(len(rotangles)):    
        chess_pts.append(get_chessboard_rot_trans(world_pts,rotangles[j][0],rotangles[j][1],rotangles[j][2],transangles[i][0],transangles[i][1],transangles[i][2]))

chess_pts_arr = np.array(chess_pts)
chess_pts_arr.shape

(50, 25, 3)

In [126]:
R = get_rotation_matrix(theta_x=THETA_X, theta_z=THETA_Z)
world_origin = np.zeros(3)
dx, dy, dz = np.eye(3)
world_frame = ReferenceFrame(
    origin=world_origin, 
    dx=dx, 
    dy=dy,
    dz=dz,
    name="World",
)
camera_frame = ReferenceFrame(
    origin=C, 
    dx=R @ dx, 
    dy=R @ dy,
    dz=R @ dz,
    name="Camera",
)
Z = PrincipalAxis(
    camera_center=camera_frame.origin,
    camera_dz=camera_frame.dz,
    f=F,
)
image_frame = ReferenceFrame(
    origin=Z.p - camera_frame.dx * PX - camera_frame.dy * PY, 
    dx=R @ dx, 
    dy=R @ dy,
    dz=R @ dz,
    name="Image",
)
image_plane = ImagePlane(
    origin=image_frame.origin, 
    dx=image_frame.dx, 
    dy=image_frame.dy, 
    heigth=IMAGE_HEIGTH,
    width=IMAGE_WIDTH,
)

In [127]:
planes_lst = []
for i in range(chess_pts_arr.shape[0]):
    planes_lst.append(get_plane(chess_pts_arr[i,:,:]))

planes_arr = np.array(planes_lst)

In [128]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(projection="3d")
ax.text(*C, "C")
world_frame.draw3d()
camera_frame.draw3d()
image_frame.draw3d()
Z.draw3d()
image_plane.draw3d()
for i in range(chess_pts_arr.shape[0]):
    x_surface, y_surface, z_surface = get_plane(chess_pts_arr[i,:,:])
    ax.plot_surface(x_surface,y_surface,z_surface)

ax.view_init(elev=30.0, azim=30.0)

ax.set_title(f"Representation of a calibration planes in 3D")
ax.set_xlabel("X-axis")
ax.set_ylabel("Y-axis")
ax.set_zlabel("Z-axis")
plt.tight_layout()
plt.show()

In [129]:
x_lst = []
homographies = []
for i in range(chess_pts_arr.shape[0]):
    x_arr, X_arr = get_image_points(chess_pts_arr[i,:,:].T,PX,PY,-np.pi/2,np.pi/2,trans_x= C[0],trans_y= C[1],trans_z= C[2],F = 1)

    x_lst.append(x_arr)

x_zhang = np.array(x_lst,np.float32)

In [130]:
patternsize_x = 4
patternsize_y = 4

pattern_size_x,pattern_size_y = (patternsize_x,patternsize_y)

X = np.linspace(0, pattern_size_x,pattern_size_x + 1)
Y = np.linspace(0, pattern_size_y,pattern_size_y + 1)

zdata = np.zeros((patternsize_x + 1,patternsize_y + 1))
xdata, ydata = np.meshgrid(X, Y)

xdata_ = xdata.flatten()
ydata_ = ydata.flatten()
zdata_ = zdata.flatten()
# homogeneous coordinates
world_pts =([xdata_,ydata_,zdata_])
world_pts_ = np.array(world_pts).T

In [131]:
world_pts_arr = np.zeros((50,25,3),np.float32)
for i in range(50):
    world_pts_arr[i,:,:] = world_pts_

world_pts_arr = world_pts_arr.reshape(50,1,25,3)

In [144]:

X_h = to_homogeneus_arr(world_pts_)
x_h = to_homogeneus_arr(x_zhang[0,:,:])

Txyz, X_h_n = normalization(X_h, 3)
Txy, x_h_n = normalization(x_h, 2)

P_n = DLT(x_h_n,X_h_n)

homographies.append(denormalization(P_n,Txy,Txyz))

print("Camera Matrix:\n",P_n)


Camera Matrix:
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]]


In [160]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(world_pts_arr, x_zhang, (11,7), None, None)

In [161]:
print("Intrisic Camera Matrix:\n" + str(mtx))
#print("Lens distortion coefficients:\n" + str(dist))
#print("Rotation vetors\n" + str(rvecs))
#print("Translation vetors\n" + str(tvecs))
print("Error\n" + str(ret))


Intrisic Camera Matrix:
[[1.33582885 0.         5.45777913]
 [0.         1.35845804 3.4273002 ]
 [0.         0.         1.        ]]
Error
0.010554146722927855


In [171]:
for i in range(len(world_pts_arr)):
    imgpoints2, _ = cv2.projectPoints(world_pts_arr[i], rvecs[0], tvecs[0], mtx, dist)
    imgpoints2 = imgpoints2.reshape(-1,2)

In [173]:
fig = plt.figure(figsize=(11,7))
ax = fig.add_subplot()
ax.scatter(imgpoints2[:,0],imgpoints2[:,1])
ax.scatter(x_zhang[0,:,0],x_zhang[0,:,1])

plt.show()

In [169]:
x_zhang[49,:,:]

array([[ 7.5669875,  4.5      ],
       [ 8.109707 ,  4.642857 ],
       [ 8.833333 ,  4.8333335],
       [ 9.84641  ,  5.1      ],
       [11.366025 ,  5.5      ],
       [ 7.5669875,  4.75     ],
       [ 8.109707 ,  4.928571 ],
       [ 8.833333 ,  5.1666665],
       [ 9.84641  ,  5.5      ],
       [11.366025 ,  6.       ],
       [ 7.5669875,  5.       ],
       [ 8.109707 ,  5.214286 ],
       [ 8.833333 ,  5.5      ],
       [ 9.84641  ,  5.9      ],
       [11.366025 ,  6.5      ],
       [ 7.5669875,  5.25     ],
       [ 8.109707 ,  5.5      ],
       [ 8.833333 ,  5.8333335],
       [ 9.84641  ,  6.3      ],
       [11.366025 ,  7.       ],
       [ 7.5669875,  5.5      ],
       [ 8.109707 ,  5.785714 ],
       [ 8.833333 ,  6.1666665],
       [ 9.84641  ,  6.7      ],
       [11.366025 ,  7.5      ]], dtype=float32)

In [177]:
tot_error = 0
total_points = 0

for i in range(len(x_zhang)):
    imgpoints2, _ = cv2.projectPoints(world_pts_arr[i], rvecs[i], tvecs[i], mtx, dist)
    imgpoints2 = imgpoints2.reshape(-1,2)
    tot_error += np.sum(np.abs(x_zhang[i]-imgpoints2)**2)
    total_points += len(x_zhang[i])
    
mean_error=np.sqrt(tot_error/total_points)
print("Mean reprojection error: ", mean_error)

Mean reprojection error:  0.010554151515071945
